# IMPORTING LIBRARIES

In [76]:
!nvidia-smi

/bin/bash: /home/aziz0220/anaconda3/envs/py310/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
Sun May  5 05:08:48 2024       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.76.01              Driver Version: 552.22         CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA GeForce RTX 4060 ...    On  |   00000000:01:00.0  On |                  N/A |
| N/A   59C    P0             20W /

In [77]:
# # Load the TensorBoard notebook extension.
# %load_ext tensorboard
# from datetime import datetime
# from packaging import version
# 
# import tensorflow as tf
# from tensorflow import keras
# print("TensorFlow version: ", tf.__version__)
# assert version.parse(tf.__version__).release[0] >= 2, \
#     "This notebook requires TensorFlow 2.0 or above."
# 
# 
# import tensorboard
# tensorboard.__version__

In [78]:
import seg_metrics.seg_metrics as sg
from skimage import io
from skimage.metrics import hausdorff_distance
import numpy as np
import cv2
from glob import glob
from tensorflow.keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping, CSVLogger
from matplotlib import pyplot as plt
import SimpleITK as sitk
import os
#from keras.utils import to_categorical
import tensorflow as tf
import keras
from PIL import Image
import shutil
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from tensorflow import keras
from tqdm import tqdm
from sklearn.metrics import f1_score, jaccard_score
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"
from tensorflow.keras.layers import Conv2D, BatchNormalization, Activation, MaxPool2D, Conv2DTranspose, Concatenate, Input
from tensorflow.keras.models import Model
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler

In [79]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_virtual_device_configuration(
                gpu,
                [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=20000)]) 
            logical_gpus = tf.config.experimental.list_logical_devices('GPU')
            configs  = tf.config.get_logical_device_configuration( gpu )  
            print(gpu, " :::::::::::: ", logical_gpus , " :::::: ", configs)
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')  ::::::::::::  [LogicalDevice(name='/device:GPU:0', device_type='GPU')]  ::::::  [LogicalDeviceConfiguration(memory_limit=20000, experimental_priority=None, experimental_device_ordinal=None)]
1 Physical GPUs, 1 Logical GPUs


# SETTING GLOBAL VARIABLES

In [80]:
global IMG_H
global IMG_W
global IMG_D
global NUM_CLASSES
global CLASSES
global image_h
global image_w
global num_classes
global classes
global rgb_codes
global COLORMAP
global case_names
global dataset_path
global input_shape
global dataOutputPath
image_h = 512
image_w = 512
num_classes = 31
input_shape = (image_h, image_w, 3)
batch_size = 2
lr = 1e-4
num_epochs = 7
HOUNSFIELD_MIN = -1000
HOUNSFIELD_MAX = 2000
HOUNSFIELD_RANGE = HOUNSFIELD_MAX - HOUNSFIELD_MIN
SLICE_DECIMATE_IDENTIFIER = 3
SLICE_X = True
SLICE_Y = False
SLICE_Z = False
dataset_path = '/mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/'
os.chdir(dataset_path)
input_path =dataset_path + 'set_2/'
case_names = [f"case_{num:02d}" for num in range(1, 43)]
dataOutputPath =dataset_path + 'set_4/'
imageSliceOutput = os.path.join(dataOutputPath, 'img/')
maskSliceOutput = os.path.join(dataOutputPath, 'mask/')
trainsetOutput = os.path.join(dataOutputPath, 'train/')
testsetOutput = os.path.join(dataOutputPath, 'test/')
dataset_path = '/mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4'
model_path = os.path.join("files", "model.h5")
csv_path = os.path.join("files", "data.csv")
SEED = 909
BATCH_SIZE_TRAIN = 2
BATCH_SIZE_TEST = 2
IMAGE_HEIGHT = 512
IMAGE_WIDTH = 512
IMG_SIZE = (IMAGE_HEIGHT, IMAGE_WIDTH)
data_dir = '/mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4/'
data_dir_train = os.path.join(data_dir, 'train/')
data_dir_train_image = os.path.join(data_dir_train, 'images/')
data_dir_train_mask = os.path.join(data_dir_train, 'masks/')
data_dir_test = os.path.join(data_dir, 'test/')
data_dir_test_image = os.path.join(data_dir_test, 'images/')
data_dir_test_mask = os.path.join(data_dir_test, 'masks/')
NUM_OF_EPOCHS = 10
LABEL_dict = {
    "background": 0,
    "A_Carotid_L": 1,
    "A_Carotid_R": 2,
    "Arytenoid": 3,
    "Bone_Mandible": 4,
    "Brainstem": 5,
    "BuccalMucosa": 6,
    "Cavity_Oral": 7,
    "Cochlea_L": 8,
    "Cochlea_R": 9,
    "Cricopharyngeus": 10,
    "Esophagus_S": 11,
    "Eye_AL": 12,
    "Eye_AR": 13,
    "Eye_PL": 14,
    "Eye_PR": 15,
    "Glnd_Lacrimal_L": 16,
    "Glnd_Lacrimal_R": 17,
    "Glnd_Submand_L": 18,
    "Glnd_Submand_R": 19,
    "Glnd_Thyroid": 20,
    "Glottis": 21,
    "Larynx_SG": 22,
    "Lips": 23,
    "OpticChiasm": 24,
    "OpticNrv_L": 25,
    "OpticNrv_R": 26,
    "Parotid_L": 27,
    "Parotid_R": 28,
    "Pituitary": 29,
    "SpinalCord": 30,
}
rgb_codes = [
    [255, 255, 255],          # Background
            [244, 214, 49],    # SpinalCord
            [216, 101, 79],    # A_Carotid_L
            [216, 101, 79],    # A_Carotid_R
            [183, 156, 220],   # Arytenoid
            [222, 198, 101],   # Bone_Mandible
            [145, 92, 109],    # Brainstem
            [178, 69, 182],    # BuccalMucosa
            [121, 39, 153],    # Cavity_Oral
            [104, 181, 63],    # Cochlea_L
            [123, 174, 91],    # Cochlea_R
            [220, 127, 211],   # Cricopharyngeus
            [174, 125, 64],    # Esophagus_S
            [127, 75, 38],     # Eye_AL
            [127, 75, 38],     # Eye_AR
            [53, 152, 174],    # Eye_PL
            [53, 152, 174],    # Eye_PR
            [86, 58, 127],     # Glnd_Lacrimal_L
            [86, 58, 127],     # Glnd_Lacrimal_R
            [222, 198, 101],   # Glnd_Submand_L
            [222, 198, 101],   # Glnd_Submand_R
            [62, 162, 114],    # Glnd_Thyroid
            [47, 210, 120],    # Glottis
            [150, 208, 243],   # Larynx_SG
            [188, 91, 95],     # Lips
            [99, 106, 24],     # OpticChiasm
            [127, 24, 70],     # OpticNrv_L
            [127, 24, 70],     # OpticNrv_R
            [31, 45, 172],     # Parotid_L
            [31, 45, 172],     # Parotid_R
            [57, 157, 110]  
]
classes = [
    "background",  
            "A_Carotid_L",
            "A_Carotid_R",
            "Arytenoid",
            "Bone_Mandible",
            "Brainstem",
            "BuccalMucosa",
            "Cavity_Oral",
            "Cochlea_L",
            "Cochlea_R",
            "Cricopharyngeus",
            "Esophagus_S",
            "Eye_AL",
            "Eye_AR",
            "Eye_PL",
            "Eye_PR",
            "Glnd_Lacrimal_L",
            "Glnd_Lacrimal_R",
            "Glnd_Submand_L",
            "Glnd_Submand_R",
            "Glnd_Thyroid",
            "Glottis",
            "Larynx_SG",
            "Lips",
            "OpticChiasm",
            "OpticNrv_L",
            "OpticNrv_R",
            "Parotid_L",
            "Parotid_R",
            "Pituitary",
            "SpinalCord"
]

# FUNCTIONS

 1. LOADING DATA FUNCTION (from one case)

In [81]:
def load_data(case_number):
    CT_volume = io.imread(input_path + f"/{case_names[case_number]}/{case_names[case_number]}_IMG_CT.nrrd")
    MR_volume = io.imread(input_path + f"/{case_names[case_number]}/{case_names[case_number]}_IMG_MR_T1.nrrd")
    mask = io.imread(input_path + f"/{case_names[case_number]}/{case_names[case_number]}_stacked_segments.seg.nrrd")
    print(f"Case number {case_number} is being processed : Mask shape = {mask.shape}  || CT shape = {CT_volume.shape} || MR shape = {MR_volume.shape}")
    return CT_volume, MR_volume, mask

In [82]:
def load_data_sitk(case_number):
    CT_volume = sitk.ReadImage(input_path + f"/{case_names[case_number]}/{case_names[case_number]}_IMG_CT.nrrd")
    MR_volume = sitk.ReadImage(input_path + f"/{case_names[case_number]}/{case_names[case_number]}_IMG_MR_T1.nrrd")
    mask = sitk.ReadImage(input_path + f"/{case_names[case_number]}/{case_names[case_number]}_stacked_segments.seg.nrrd")
    return CT_volume, MR_volume, mask

 2. NORMALIZING INTENSITY FUNCTION

In [83]:
def normalizeImageIntensityRange(img, clip_min=0, shiftThreshold=1):
    minimg=min(img[img > 0].flatten())
    #print(f"Min value is {minimg}, Max value is {max(img.flatten())}, dtype is {img.dtype}")
    shifted_img = img.copy()  # Create a copy to avoid in-place modification
    if(minimg > shiftThreshold):  
        shifted_img = shifted_img.astype(np.int16)
        shifted_img -= minimg
        #print(f"Min value is {min(shifted_img.flatten())}, Max value is {max(shifted_img.flatten())}")
        shifted_img = np.clip(shifted_img, clip_min, None)
        #print(f"Min value is {min(shifted_img.flatten())},Max value is {max(shifted_img.flatten())}, MR SHAPE : {shifted_img.shape} ")
    return shifted_img

3. PLOTTING INDIVIDIAL CLASSES FUNCTION

In [84]:
def plot_individual_classes(mask, LABEL_dict):
  # Get unique pixel values and counts (excluding background)
  unique_values, counts = np.unique(mask, return_counts=True)

  # Iterate through unique values (excluding background)
  plt.imshow(mask, cmap='gray')
  plt.title("Full Mask")
  plt.axis('off')
  plt.show()  # Display the plot
  plt.close()  # Close the plot window after display

  for i, class_value in enumerate(unique_values):
    # Create a binary mask for the current class
    class_mask = np.zeros_like(mask)
    class_mask[mask == class_value] = 255  # Set the pixels of the current class to white
    print(class_value)
    # Get class name from LABEL_dict (handling potential missing keys)
    class_name = [key for key, value in LABEL_dict.items() if value == class_value][0]
    # Plot the class mask
    plt.imshow(class_mask, cmap='gray')
    plt.title(f"{class_name} (Count: {counts[i]})")
    plt.axis('off')
    plt.show()  # Display the plot
    plt.close()  # Close the plot window after display


CENTERING MR AND CT


In [85]:
def center_crop_volumes(case_number):
    ct_image, mri_image, mask=load_data_sitk(case_number)
    #mask = sitk.Cast(sitk.RescaleIntensity(mask), sitk.sitkUInt8)
    mr_resampled = sitk.Resample(mri_image, ct_image)
    ct_array = sitk.GetArrayFromImage(ct_image)
    mri_array = sitk.GetArrayFromImage(mr_resampled)
    mask_array = sitk.GetArrayFromImage(mask)
    #scaled_mask = (mask_array / 255 * 31)  # Perform scaling
    #mask_uint8 = scaled_mask.astype(np.uint8)
    return ct_array, mri_array, mask_array

4. SAVING SLICE FUNCTION

In [86]:
def saveSlice(img, fname, path, mode='image'):
  os.makedirs(path, exist_ok=True)  # Ensure path exists
  #dtyper = str(img.dtype)  # Store original dtype
  img = np.flipud(img)
  image = Image.fromarray(img) 
  new_size = max(img.shape[0], img.shape[1])
  image = image.resize((new_size, new_size), Image.NEAREST)
  width, height = 512, 512  # Desired crop dimensions
  left = (image.width - width) // 2
  top = (image.height - height) // 2
  right = left + width
  bottom = top + height
  image = image.crop((left, top, right, bottom))
  img = np.array(image)
  fgs=Image.fromarray(img)
  fout = os.path.join(path, f'{fname}.png')
  if(mode == 'mask'):
    fgs.save(fout)
  else:
    plt.imsave(fout,fgs,cmap='gray' )
  if( is_empty_image(img) and mode == 'image'):
    print(f"Empty image {fout}")
  return img

5. SLICING AND SAVING VOLUME FUNCTION

In [87]:
def sliceAndSaveVolumeImage(vol, mask, fname, pathimg, pathmask, class_counts, label_dict=LABEL_dict, mode='image'):
    (dimx, dimy, dimz) = vol.shape
    cnt = 0
    empty =0
    if SLICE_X:
        cnt += dimx
        #print('Slicing X: ')
        for i in range(dimx):
            if(not is_empty_image(vol[i,:,:])):
                img=saveSlice(vol[i,:,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_x', pathimg, mode='image')
                msk=saveSlice(mask[i,:,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_x', pathmask, mode='mask')
                class_counts = class_to_dict(msk, class_counts, label_dict) 
            else:
                empty+=1
    if SLICE_Y:
        cnt += dimy
        #print('Slicing Y: ')
        for i in range(dimy):
            if(not is_empty_image(vol[:,i,:])):
                img=saveSlice(vol[:,i,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_y', pathimg, mode='image')
                msk=saveSlice(mask[:,i,:], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_y', pathmask, mode='mask')
                class_counts = class_to_dict(msk, class_counts, label_dict)     
            else:
                empty+=1            
    if SLICE_Z:
        cnt += dimz
        #print('Slicing Z: ')
        for i in range(dimz):
            if(not is_empty_image(vol[:,:,i])):
                img=saveSlice(vol[:,:,i], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_z', pathimg, mode='image')
                msk=saveSlice(mask[:,:,i], fname+f'-slice{str(i).zfill(SLICE_DECIMATE_IDENTIFIER)}_z', pathmask, mode='mask')
                class_counts = class_to_dict(msk, class_counts, label_dict)                       
            else:
                empty+=1
    print(f"Removed {empty} Empty slices out of {cnt} total slices")
    cnt = cnt - empty
    return cnt

6. IS IMAGE EMPTY THRESHOLD FUNCTION

In [88]:
def is_empty_image_combined(image_path, threshold=10):
  with Image.open(image_path) as img:
    # Convert to grayscale
    grayscale_img = img.convert('L')
    grayscale_img = np.array(grayscale_img)
    uniform = np.all(np.unique(grayscale_img) == grayscale_img[0, 0])
    # Check non-zero pixel sum (avoids black image misclassification)
    non_zero_sum = np.sum(grayscale_img != 0)
    print(f"Uniform: {uniform}, Non-zero sum: {non_zero_sum}")
    return uniform and (non_zero_sum < threshold)

IS EMPTY IMAGE FUNCTION

In [89]:
def is_empty_image(img):
    return np.all(img == 0) 

7. CROP DATA FUNCTION

In [90]:
def crop_data(CT_volume, MR_volume, mask, crop_type='center'):

  # Get minimum overlap dimensions
  min_shape = (min(CT_volume.shape[0], MR_volume.shape[0]),
               min(CT_volume.shape[1], MR_volume.shape[1]),
               min(CT_volume.shape[2], MR_volume.shape[2]))

  # Handle different cropping types
  if crop_type == 'center':
    # Calculate starting indices for centered cropping
    start_CT_x = int((CT_volume.shape[1] - min_shape[1]) / 2)
    start_CT_y = int((CT_volume.shape[0] - min_shape[0]) / 2)
    start_CT_z = int((CT_volume.shape[2] - min_shape[2]) / 2)

    start_MR_x = int((MR_volume.shape[1] - min_shape[1]) / 2)
    start_MR_y = int((MR_volume.shape[0] - min_shape[0]) / 2)
    start_MR_z = int((MR_volume.shape[2] - min_shape[2]) / 2)
    
    start_MK_x = int((mask.shape[1] - min_shape[1]) / 2)
    start_MK_y = int((mask.shape[0] - min_shape[0]) / 2)
    start_MK_z = int((mask.shape[2] - min_shape[2]) / 2)

    # Crop volumes from center
    cropped_CT = CT_volume[start_CT_y:start_CT_y + min_shape[0],
                             start_CT_x:start_CT_x + min_shape[1],
                             start_CT_z:start_CT_z + min_shape[2]]
    cropped_MR = MR_volume[start_MR_y:start_MR_y + min_shape[0],
                             start_MR_x:start_MR_x + min_shape[1],
                             start_MR_z:start_MR_z + min_shape[2]]
    cropped_mask = mask[start_MK_y:start_MK_y + min_shape[0],
                                start_MK_x:start_MK_x + min_shape[1],
                                start_MK_z:start_MK_z + min_shape[2]]   
    #print(f"CT shape: {cropped_CT.shape}, MR shape: {cropped_MR.shape}, Mask shape: {cropped_mask.shape}")
  elif crop_type == 'min':
    # Crop minimum overlap region (might leave black borders)
    # Implement logic for minimum overlap cropping here (consider using np.where)
    pass
  elif crop_type == 'max':
    # Crop maximum overlap region (might cut off some data)
    # Implement logic for maximum overlap cropping here (consider slicing)
    pass
  else:
    raise ValueError(f"Invalid crop_type: {crop_type}. Supported options are 'center', 'min', and 'max'.")
    
  return cropped_CT, cropped_MR, cropped_mask


In [91]:
def crop_data_v2(CT_volume, MR_volume, mask, original_CT, original_MR, crop_type='center'):

  # Get minimum overlap dimensions
  min_shape = (min(original_CT[0], original_MR[0]),
               min(original_CT[1], original_MR[1]),
               min(original_CT[2], original_MR[2]))

  # Handle different cropping types
  if crop_type == 'center':
    unique_classes, counts = np.unique(mask, return_counts=True)
    print(len(unique_classes))
    # Calculate starting indices for centered cropping
    start_CT_x = int((CT_volume.shape[1] - min_shape[1]) / 2)
    start_CT_y = int((CT_volume.shape[0] - min_shape[0]) / 2)
    start_CT_z = int((CT_volume.shape[2] - min_shape[2]) / 2)

    start_MR_x = int((MR_volume.shape[1] - min_shape[1]) / 2)
    start_MR_y = int((MR_volume.shape[0] - min_shape[0]) / 2)
    start_MR_z = int((MR_volume.shape[2] - min_shape[2]) / 2)
    
    start_MK_x = int((mask.shape[1] - min_shape[1]) / 2)
    start_MK_y = int((mask.shape[0] - min_shape[0]) / 2)
    start_MK_z = int((mask.shape[2] - min_shape[2]) / 2)

    # Crop volumes from center
    cropped_CT = CT_volume[start_CT_y:start_CT_y + min_shape[0],
                             start_CT_x:start_CT_x + min_shape[1],
                             start_CT_z:start_CT_z + min_shape[2]]
    cropped_MR = MR_volume[start_MR_y:start_MR_y + min_shape[0],
                             start_MR_x:start_MR_x + min_shape[1],
                             start_MR_z:start_MR_z + min_shape[2]]
    cropped_mask = mask[start_MK_y:start_MK_y + min_shape[0],
                                start_MK_x:start_MK_x + min_shape[1],
                                start_MK_z:start_MK_z + min_shape[2]]   
    unique_classes, counts = np.unique(cropped_mask, return_counts=True)
    print(len(unique_classes))
    #print(f"CT shape: {cropped_CT.shape}, MR shape: {cropped_MR.shape}, Mask shape: {cropped_mask.shape}")
  elif crop_type == 'min':
    # Crop minimum overlap region (might leave black borders)
    # Implement logic for minimum overlap cropping here (consider using np.where)
    pass
  elif crop_type == 'max':
    # Crop maximum overlap region (might cut off some data)
    # Implement logic for maximum overlap cropping here (consider slicing)
    pass
  else:
    raise ValueError(f"Invalid crop_type: {crop_type}. Supported options are 'center', 'min', and 'max'.")
    
  return cropped_CT, cropped_MR, cropped_mask


8. CONCATENATE DATA FUNCTION

In [92]:
# def concatenate_data(concatenated_volume ,cropped_ct_image, MR_volume, Masks):
#     if(np.all(Masks == 0) and np.all(concatenated_volume == 0)):
#         concatenated_volume = np.concatenate((cropped_ct_image, MR_volume), axis=0)
#         Masks = np.concatenate((cropped_mask, cropped_mask), axis=0)
#         #print(f"Volume shape is {concatenated_volume.shape}  ")
#         return concatenated_volume, Masks
#     concatenated_volume = np.concatenate((concatenated_volume, cropped_ct_image), axis=0)
#     concatenated_volume = np.concatenate((concatenated_volume, MR_volume), axis=0)
#     Masks = np.concatenate((Masks, cropped_mask), axis=0)
#     Masks = np.concatenate((Masks, cropped_mask), axis=0)
#     #print(f"Volume shape is {concatenated_volume.shape} ")
#     if(concatenated_volume.shape != Masks.shape):
#         print("Concatenation error")
#     return concatenated_volume, Masks

9. EXTRACTING CLASSES NUMBER FROM MASK FUNCTION

In [93]:
def class_to_dict (msk, class_counts, label_dict = LABEL_dict):
    unique_classes, counts = np.unique(msk, return_counts=True)
    for class_value, count in zip(unique_classes, counts):
        if class_value in label_dict.values():
            class_name = [key for key, value in label_dict.items() if value == class_value][0]
            class_counts[class_name] += 1
    return class_counts

10. LOADING ALL DATA INTO SLICES FUNCTION

In [94]:
def load_all_data(start_case = 0, end_case = len(case_names), normalize=False, concatenate=False , target_shape = (512, 512, 512), label_dict = LABEL_dict, imageSliceOutput = imageSliceOutput, maskSliceOutput = maskSliceOutput):
    concatenated_volume = np.empty((512, 512, 512))
    numOfSlicesCT = 0
    numOfSlicesMR = 0
    if concatenate:
        Masks = np.empty((512, 512, 512))
    class_counts = {name: 0 for name in label_dict.keys()}  
    for case_number in range(start_case, end_case):
        CT_volume_old, MR_volume_old, mask_old= load_data(case_number)      
        original_CT = CT_volume_old.shape
        original_MR = MR_volume_old.shape
        CT_volume, MR_volume, mask = center_crop_volumes(case_number)
        #cropped_ct_image, cropped_mr_image, cropped_mask =crop_data_v2(CT_volume, MR_volume, mask, original_CT, original_MR)   
        cropped_ct_image, cropped_mr_image, cropped_mask = CT_volume, MR_volume, mask
        if normalize:
            cropped_mr_image=normalizeImageIntensityRange(cropped_mr_image)
            #cropped_ct_image=normalizeImageIntensityRange(cropped_ct_image)
        if concatenate:
            # concatenated_volume, Masks = concatenate_data(concatenated_volume, cropped_ct_image, cropped_mr_image, cropped_mask)
            if(concatenated_volume.shape[0] > target_shape[0]):
                concatenated_volume = concatenated_volume[:input_shape[0]]
                Masks = Masks[:input_shape[0]]
            if(start_case == start_case):
                numOfSlicesCTMR = sliceAndSaveVolumeImage(concatenated_volume, f"case_{case_number}_MR_CT", imageSliceOutput)
            else :
                numOfSlicesCTMR = sliceAndSaveVolumeImage(concatenated_volume, f"case_{start_case}_{case_number}_MR_CT", imageSliceOutput)
            print(f'\n {numOfSlicesCTMR} CT + MR slices created \n')
            numOfSlicesMask = sliceAndSaveVolumeImage(Masks, f"case_{case_number}_mask", maskSliceOutput)
            print(f'\n {numOfSlicesMask} mask slices created \n')
            return case_number
        else:
            numOfSlicesCT += sliceAndSaveVolumeImage(cropped_ct_image,cropped_mask, f"case_{case_number}_CT", imageSliceOutput, maskSliceOutput,class_counts = class_counts)
            # numOfSlicesMR += sliceAndSaveVolumeImage(cropped_mr_image, cropped_mask, f"case_{case_number}_MR", imageSliceOutput, maskSliceOutput, class_counts = class_counts, mode='imageMR')
    print(f'\n {numOfSlicesMR} MR slices created \n')
    print(f'\n {numOfSlicesCT} CT slices created \n')
    print(f'\n {numOfSlicesCT+numOfSlicesMR} mask slices created \n')
    print(f"\n Class counts: {class_counts} \n")
    return end_case

11. COUNTING EMPTY MASKS FUNCTION

In [95]:
def count_empty_masks(mask_folder):
  """
  Counts the number of empty masks in a given folder.

  Args:
      mask_folder (str): The path to the folder containing masks.

  Returns:
      tuple: A tuple containing the number of empty masks and total masks.
  """

  total_masks = 0
  empty_masks = 0

  for filename in os.listdir(mask_folder):
    if filename.endswith('.png'):  # Adjust for your mask format
      mask_path = os.path.join(mask_folder, filename)
      mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
      #mask_img = Image.open(mask_path)
      #mask = np.array(mask_img)
      # Check if all pixels are black (empty mask)
      if np.all(mask == 0):
        empty_masks += 1
    total_masks += 1

  return empty_masks, total_masks

12. GET NON EMPTY IMAGES FUNCTION

In [96]:
def get_non_empty_images(image_path):
  if not os.path.isdir(image_path):
    raise ValueError("Provided path is not a directory")
  non_empty_images = []
  # Loop through all files in the directory
  for filename in os.listdir(image_path):
    if filename.lower().endswith((".png")):
      full_path = os.path.join(image_path, filename)
      mask = cv2.imread(full_path, cv2.IMREAD_GRAYSCALE) 
      #mask_img = Image.open(full_path)
      #mask = np.array(mask_img)
      # Check if all pixels are black (empty mask)
      if not np.all(mask == 0):
        non_empty_images.append(filename)
  return non_empty_images

13. COUNT MASK CLASSES FROM PATH FUNCTION

In [97]:
def count_classes_from_mask(mask_folder, filename):
    if filename.endswith('.png'):
        mask_path = os.path.join(mask_folder, filename)
        mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
        num_classes = count_mask_classes(mask)
    return num_classes

14. COUNT MASK CLASSES FUNCTION

In [98]:
def count_mask_classes(mask):
  foreground_mask = mask[mask >= 0]
  # Count the number of unique values (including background)
  num_classes = len(np.unique(foreground_mask))
  print(f"Number of classes: {num_classes}")
  return num_classes

COUNT MASK CLASSES PER CLASS from path

In [99]:
def count_mask_per_class( path, filenames, label_dict = LABEL_dict): 
  class_counts = {name: 0 for name in label_dict.keys()}
  for filename in filenames:
    if filename.endswith('.png'):
      mask_path = os.path.join(path, filename)
      mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
      #mask = mask.astype(np.float32) / 255.0
      class_counts = class_to_dict(mask, class_counts, label_dict)                 
  return class_counts

COUNT MASKs CLASSES from path

In [100]:
def count_masks_per_class(mask_dir, label_dict):
  class_counts = {name: 0 for name in label_dict.keys()}
  for filename in os.listdir(mask_dir):
    if filename.endswith('.png'):
      mask_path = os.path.join(mask_dir, filename)
      mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
      class_counts = class_to_dict(mask, class_counts, label_dict)         
      #class_counts = delete_zero_count_classes(class_counts)
  return class_counts

In [101]:
def delete_zero_count_classes(class_counts):
  return {key: value for key, value in class_counts.items() if value > 0}

In [102]:
def extract_class_masks(mask_folder_path, label_dict):
  for filename in os.listdir(mask_folder_path):
    if filename.lower().endswith(".png"):  # Check for PNG format
      mask_path = os.path.join(mask_folder_path, filename)
      mask_img = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)  # Read as grayscale
      unique_classes, counts = np.unique(mask_img, return_counts=True)
      for class_value, count in zip(unique_classes, counts):
        if class_value in label_dict.values():
            class_name = [key for key, value in label_dict.items() if value == class_value][0]
            class_mask = np.zeros_like(mask_img, dtype=np.uint8) 
            class_mask[mask_img == class_value] = 255
            class_folder_path = os.path.join(mask_folder_path, class_name)
            if not os.path.exists(class_folder_path):
              os.makedirs(class_folder_path)
            # Save the extracted class mask with the original filename
            class_mask_path = os.path.join(class_folder_path, filename)
            cv2.imwrite(class_mask_path, class_mask)

In [103]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [104]:
def load_dataset(path):
    train_x = sorted(glob(os.path.join(path, "train", "images/images", "*.png")))
    train_y = sorted(glob(os.path.join(path, "train", "masks/masks", "*.png")))

    valid_x = sorted(glob(os.path.join(path, "val", "images/images", "*.png")))
    valid_y = sorted(glob(os.path.join(path, "val", "masks/masks", "*.png")))

    test_x = sorted(glob(os.path.join(path, "test", "images/images", "*.png")))
    test_y = sorted(glob(os.path.join(path, "test", "masks/masks", "*.png")))

    return (train_x, train_y), (valid_x, valid_y), (test_x, test_y)

In [105]:
def read_image_mask(x, y):
    """ Image """
    x = cv2.imread(x, cv2.IMREAD_COLOR)
    #x = cv2.resize(x, (image_w, image_h))
    #x = 1-x 
    x = x / 255.0
    x = x.astype(np.float32)
    #x = x.astype(np.uint8)
    #x = np.stack((x,)*3, axis=-1)
    #print(x.shape, x.dtype)
    """ Mask """
    y = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
    #y = cv2.resize(y, (image_w, image_h))
    y = y.astype(np.int32)
    
    #x, y = random_flip_left_right(x, y)  # Using TensorFlow's augmentation
    #x, y = data_aug(x[np.newaxis, ...], y[np.newaxis, ...])[0]
    return x, y

In [106]:
data_aug = ImageDataGenerator(
    rotation_range=90,  # Randomly rotate images by up to 20 degrees
    width_shift_range=0.2,  # Randomly shift images horizontally by up to 20% of their width
    height_shift_range=0.2,  # Randomly shift images vertically by up to 20% of their height
    shear_range=0.2,  # Randomly apply shearing transformation
    zoom_range=0.2,  # Randomly zoom images in or out by up to 20%
    horizontal_flip=True  # Randomly flip images horizontally with a 50% chance
)


In [107]:
def preprocess(x, y):
    def f(x, y):
        x = x.decode()
        y = y.decode()
        return read_image_mask(x, y)
    
    image, mask = tf.numpy_function(f, [x, y], [tf.float32, tf.int32])
    mask = tf.one_hot(mask, num_classes)
    image.set_shape([image_h, image_w, 3])
    mask.set_shape([image_h, image_w, num_classes])

    return image, mask

In [108]:
class Augment(tf.keras.layers.Layer):
  def __init__(self, seed=42):
    super().__init__()
    self.seed = seed
    
    self.augment_inputs = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
    self.augment_labels = tf.keras.layers.RandomFlip(mode="horizontal", seed=seed)
    
    self.rotate_inputs = tf.keras.layers.RandomRotation(factor=0.2, seed=self.seed)
    self.rotate_labels = tf.keras.layers.RandomRotation(factor=0.2, seed=self.seed)
  def call(self, inputs, labels):
    # Apply horizontal flip first
    inputs = self.augment_inputs(inputs)
    labels = self.augment_labels(labels)

    # Apply random rotation afterwards (on the flipped images)
    inputs = self.rotate_inputs(inputs)
    labels = self.rotate_labels(labels)

    return inputs, labels


In [109]:
def tf_dataset_train(X, Y, batch=8):
    ds = tf.data.Dataset.from_tensor_slices((X, Y))
    ds = ds.shuffle(buffer_size=1000).map(preprocess)
    ds = ds.batch(batch).prefetch(2)
    ds = ds.map(Augment())
    return ds

In [110]:
def tf_dataset_test(X, Y, batch=8):
    ds = tf.data.Dataset.from_tensor_slices((X, Y))
    ds = ds.shuffle(buffer_size=1000).map(preprocess)
    ds = ds.batch(batch).prefetch(2)
    ds = ds.map(Augment())
    return ds

In [111]:
def grayscale_to_rgb(mask, rgb_codes):
    h, w = mask.shape[0], mask.shape[1]
    mask = mask.astype(np.int32)
    output = []
    for i, pixel in enumerate(mask.flatten()):
        output.append(rgb_codes[pixel])
    output = np.reshape(output, (h, w, 3))
    return output

In [112]:
def save_results(image_x, mask, pred, save_image_path):
    mask = np.expand_dims(mask, axis=-1)
    mask = grayscale_to_rgb(mask, rgb_codes)
    pred = np.expand_dims(pred, axis=-1)
    pred = grayscale_to_rgb(pred, rgb_codes)
    if mask.dtype != np.uint8:
        mask = mask.astype(np.uint8)
    if pred.dtype != np.uint8:
        pred = pred.astype(np.uint8)

    font = cv2.FONT_HERSHEY_SIMPLEX  # Choose a suitable font
    font_scale = 0.5
    font_thickness = 1
    text_color = (0, 0, 0)  # Black text color

    # Add labels to each image component
    image_x_label = cv2.putText(image_x.copy(), "Image", (10, 20), font, font_scale, (255, 255, 255) , font_thickness)
    mask_label = cv2.putText(mask, "Mask", (10, 20), font, font_scale, text_color, font_thickness)
    pred_label = cv2.putText(pred, "Prediction", (10, 20), font, font_scale, text_color, font_thickness)

    line = np.ones((image_x.shape[0], 10, 3)) * 255

    cat_images = np.concatenate([image_x_label, line, mask_label, line, pred_label], axis=1)
    cv2.imwrite(save_image_path, cat_images)


In [113]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters, (2, 2), strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip])
    x = conv_block(x, num_filters)
    return x

def build_unet(input_shape, num_classes):
    inputs = Input(input_shape)
    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(num_classes, 1, padding="same", activation="softmax")(d4)

    model = Model(inputs, outputs)
    return model


In [114]:
def plot_image_mask(image_path, mask_path):
  """
  Plots an image and its corresponding mask superimposed on each other.

  Args:
      image_path (str): Path to the image file.
      mask_path (str): Path to the mask file.
  """

  # Read the image and mask
  image = cv2.imread(image_path)
  mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

  # Resize the mask to match the image size if necessary
  if mask.shape[:2] != image.shape[:2]:
    mask = cv2.resize(mask, dsize=image.shape[:2], interpolation=cv2.INTER_NEAREST)
  # Apply transparency to the mask (adjust alpha for desired opacity)
  alpha = 0.5  # Set transparency level (0 for transparent, 1 for opaque)
  mask_rgb = cv2.cvtColor(mask, cv2.COLOR_GRAY2BGR)  # Convert to RGB for transparency
  mask_rgb[:, :, 2] = mask_rgb[:, :, 2] * alpha  # Set blue channel for transparency

  # Superimpose the mask on the image
  superimposed_image = cv2.addWeighted(image,0.5, mask_rgb, 1, 0)

  # Plot the image and mask
  plt.figure(figsize=(10, 5))
  plt.subplot(1, 2, 1)
  plt.imshow(image)
  plt.title("Original Image")
  plt.axis('off')

  plt.subplot(1, 2, 2)
  plt.imshow(superimposed_image)
  plt.title("Image with Mask (Superimposed)")
  plt.axis('off')

  plt.show()

#DATA AUGMENTATION

In [115]:
def create_segmentation_generator_train(img_path, msk_path, BATCH_SIZE):
    data_gen_args = dict(
                    rescale=1./255,
                       featurewise_center=True,
                      featurewise_std_normalization=True,
                      rotation_range=90,
                      width_shift_range=0.2,
                       height_shift_range=0.2,
                      zoom_range=0.2,
                        )
    datagen = ImageDataGenerator(**data_gen_args)
    
    img_generator = datagen.flow_from_directory(img_path, target_size=IMG_SIZE, class_mode=None, color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)
    msk_generator = datagen.flow_from_directory(msk_path, target_size=IMG_SIZE, class_mode=None, color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)
    return zip(img_generator, msk_generator)

In [116]:
# Remember not to perform any image augmentation in the test generator!
def create_segmentation_generator_test(img_path, msk_path, BATCH_SIZE):
    data_gen_args = dict(
         #              rescale=1./255
                        )
    datagen = ImageDataGenerator(**data_gen_args)
    
    img_generator = datagen.flow_from_directory(img_path, target_size=IMG_SIZE, class_mode=None, color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)
    msk_generator = datagen.flow_from_directory(msk_path, target_size=IMG_SIZE, class_mode=None, color_mode='grayscale', batch_size=BATCH_SIZE, seed=SEED)
    return zip(img_generator, msk_generator)

In [117]:
def display(display_list):
    plt.figure(figsize=(15,15))
    
    title = ['Input Image', 'True Mask', 'Predicted Mask']
    
    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]), cmap='gray')
    plt.show()

In [118]:
def show_dataset(datagen, num=1):
    for i in range(0,num):
        image,mask = next(datagen)
        display([image[0], mask[0]])

In [119]:
def unet(n_levels, initial_features=32, n_blocks=2, kernel_size=3, pooling_size=2, in_channels=1, out_channels=31):
    inputs = keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, in_channels))
    x = inputs
    
    convpars = dict(kernel_size=kernel_size, activation='relu', padding='same')
    
    #downstream
    skips = {}
    for level in range(n_levels):
        for _ in range(n_blocks):
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)
        if level < n_levels - 1:
            skips[level] = x
            x = keras.layers.MaxPool2D(pooling_size)(x)
            
    # upstream
    for level in reversed(range(n_levels-1)):
        x = keras.layers.Conv2DTranspose(initial_features * 2 ** level, strides=pooling_size, **convpars)(x)
        x = keras.layers.Concatenate()([x, skips[level]])
        for _ in range(n_blocks):
            x = keras.layers.Conv2D(initial_features * 2 ** level, **convpars)(x)
    # output (modified for 31 classes)
    activation = 'softmax'  # Use softmax for multi-class segmentation
    x = keras.layers.Conv2D(out_channels, kernel_size=1, activation=activation, padding='same')(x)
    return keras.Model(inputs=[inputs], outputs=[x], name=f'UNET-L{n_levels}-F{initial_features}')


In [120]:
def load_or_build_unet(model_path, input_shape, num_classes):

  try:
    # Try to load the model from the specified path
    model = tf.keras.models.load_model(model_path)
    print(f"Loaded existing model from {model_path}")
  except OSError:
    # If the file doesn't exist, build a new U-Net model
    print(f"Model not found at {model_path}. Building a new model...")
    print(f"Input shape: {input_shape}, Number of classes: {num_classes}")
    model = build_unet(input_shape, num_classes)

  return model


In [121]:
def predict_case(model, nrrd_path, x_start=None, x_end=None, y_start=None, 
                  y_end=None, z_start=None, z_end=None, output_path=None):


  # Read the nrrd data and header
  data, header = nrrd.read(nrrd_path)

  # Extract data dimensions
  data_shape = data.shape

  # Define slicing ranges (use full volume if not specified)
  x_slice = slice(x_start if x_start is not None else 0, 
                  x_end if x_end is not None else data_shape[0])
  y_slice = slice(y_start if y_start is not None else 0, 
                  y_end if y_end is not None else data_shape[1])
  z_slice = slice(z_start if z_start is not None else 0, 
                  z_end if z_end is not None else data_shape[2])

  # Slice the data (assumes channel as the last dimension)
  sliced_data = data[x_slice, y_slice, z_slice, ...]

  # Prepare empty list for predictions
  predictions = []

  # Loop through slices and predict with the model
  for image_slice in sliced_data:

    #image_slice = np.expand_dims(image_slice, axis=-1)
    #image_slice = resize_image(image_slice, model.input_shape[1], model.input_shape[2])  # Add resize function
    img = np.flipud(image_slice)
    image = Image.fromarray(img) 
    new_size = max(img.shape[0], img.shape[1])
    image = image.resize((new_size, new_size), Image.NEAREST)
    width, height = 512, 512  # Desired crop dimensions
    left = (image.width - width) // 2
    top = (image.height - height) // 2
    right = left + width
    bottom = top + height
    image = image.crop((left, top, right, bottom))
    img = np.array(image)
    image = img/255.0
    image = np.stack((image,)*3, axis=-1)  ## (H, W, 3)
    #image = np.expand_dims(image, axis=0) ## [1, H, W, 3]
    image_slice = image.astype(np.float32)

    # Make prediction on the resized slice
    predicted_slice = model.predict(image_slice[np.newaxis, ...], verbose=0)[0]
    #pred = model.predict(image, verbose=0)[0]
    # Extract the predicted class (assuming model outputs class probabilities)
    predicted_slice = np.argmax(predicted_slice[0], axis=-1)
    predicted_slice = predicted_slice.astype(np.int32)
    predictions.append(predicted_slice)

  # Combine predictions back into a single volume
  predicted_volume = np.stack(predictions, axis=0)


    
  # Update header for predicted data (assuming same data type)
  predicted_header = header.copy()
  predicted_header['data_file'] = None  # Avoid overwriting original data
  predicted_header['data_type'] = nrrd.types[predicted_volume.dtype.name]



  # Define output path if not provided
  if output_path is None:
    output_path = nrrd_path.replace('.nrrd', '_pred.nrrd')

  # Save the predicted nrrd file
  nrrd.write(output_path, predicted_volume, predicted_header)

  print(f"Prediction saved to: {output_path}")


In [122]:
#data_dir+'files_2/model_2_epoch.h5'

In [123]:
#model=tf.keras.models.load_model(data_dir+'files_2/model_2_epoch.h5')

In [124]:
#CT_path = input_path+'case_01/case_01_IMG_CT.nrrd'

In [125]:
#predict_case(model, CT_path, output_path='data_dir')

# STEP 1 : DATA PREPROCESSING

 VOLUMES TO SLICES IMAGES

In [116]:
# looping through all the cases and transforming the data from 3d into 2d slices (512 x 512) from all directions 
start_case = 0
end_case = 42
while(start_case != end_case):
    start_case=load_all_data(start_case = start_case, end_case = end_case, normalize=False)

Case number 0 is being processed : Mask shape = (202, 1024, 1024)  || CT shape = (202, 1024, 1024) || MR shape = (83, 512, 512)


KeyboardInterrupt: 

In [ ]:
#class_counts = {name: 0 for name in LABEL_dict.keys()}
#filename = 'case_0_CT-slice149_x.png'

#if filename.endswith('.png'):
#  mask_path = os.path.join(dataset_path+"/mask", filename)
#  mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
#  class_counts = class_to_dict(mask, class_counts, LABEL_dict)         
#print(class_counts)

In [ ]:
#msk = cv2.imread(maskSliceOutput + 'case_0_CT-slice149_x.png', cv2.IMREAD_GRAYSCALE)
#unique_classes, counts = np.unique(msk, return_counts=True)
#print(unique_classes)
#plot_individual_classes(msk, LABEL_dict)

In [ ]:
#image_path = imageSliceOutput  + "case_0_MR-slice434_y.png"
#mask_path = maskSliceOutput+"case_0_MR-slice434_y.png"
#plot_image_mask(image_path, mask_path)


EMPTY DATA ANALYSIS / REMOVE EMPTY DATA

In [ ]:
# empty_count, total_count = count_empty_masks(imageSliceOutput)
# 
# if total_count > 0:
#   empty_percentage = (empty_count / total_count) * 100
#   print(f"There are {empty_count} empty images out of {total_count} total images.")
#   print(f"This represents an empty images percentage of {empty_percentage:.2f}%.")
# else:
#   print("No image found in the specified folder.")
# 
# # Example usage
# empty_count, total_count = count_empty_masks(maskSliceOutput)
# 
# if total_count > 0:
#   empty_percentage = (empty_count / total_count) * 100
#   print(f"There are {empty_count} empty masks out of {total_count} total masks.")
#   print(f"This represents an empty mask percentage of {empty_percentage:.2f}%.")
# else:
#   print("No masks found in the specified folder.")


In [ ]:
non_empty_images=get_non_empty_images(imageSliceOutput)
non_empty_masks=get_non_empty_images(maskSliceOutput)
non_empty_dataset = set(non_empty_images) & set(non_empty_masks)

# STEP 2 : Splitting Dataset

In [ ]:
image_files = list(non_empty_dataset)
mask_files = image_files

image_train, image_test, mask_train, mask_test = train_test_split(image_files, mask_files, test_size=0.2, random_state=42)


# Move the corresponding image and mask files to the training and testing directories
for file_name in image_train :
    shutil.move(os.path.join(imageSliceOutput, file_name), trainsetOutput + 'images/images')
for file_name in mask_train:
    shutil.move(os.path.join(maskSliceOutput, file_name), trainsetOutput + 'masks/masks')
for file_name in image_test:
    shutil.move(os.path.join(imageSliceOutput, file_name), testsetOutput + 'images/images')
for file_name in mask_test:
    shutil.move(os.path.join(maskSliceOutput, file_name), testsetOutput + 'masks/masks')

ANALISING empty masks in the training set

In [ ]:
# # Example usage
# empty_count, total_count = count_empty_masks( trainsetOutput + '/masks/masks')
# 
# if total_count > 0:
#   empty_percentage = (empty_count / total_count) * 100
#   print(f"There are {empty_count} empty masks out of {total_count} total masks.")
#   print(f"This represents an empty mask percentage of {empty_percentage:.2f}%.")
# else:
#   print("No masks found in the specified folder.")


 each mask in his folder

In [ ]:
#extract_class_masks(trainsetOutput + '/masks/masks', LABEL_dict)
#extract_class_masks(testsetOutput + '/masks/masks', LABEL_dict)

ANALYSING CLASS EMBALANCES

In [ ]:
# train_classes = count_masks_per_class(data_dir_train_mask+ 'masks/', LABEL_dict)
# test_classes = count_masks_per_class(data_dir_test_mask + 'masks/', LABEL_dict)
# 
# print("Train set contains the following classes :",train_classes)
# print("Test set contains the following classes :", test_classes)

BLANCING CLASSES ALGORITHM

In [ ]:
def filename_classes(mask_dir, label_dict):
  class_counts = {name: [] for name in label_dict.keys()}
  for filename in os.listdir(mask_dir):
    if filename.endswith('.png'):
      mask_path = os.path.join(mask_dir, filename)
      mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
      class_counts = all_masks_classes(mask, filename, class_counts, label_dict)  # Update the actual class_counts dictionary

  return class_counts


def all_masks_classes(msk, filename,class_counts, label_dict=LABEL_dict):
    unique_classes, counts = np.unique(msk, return_counts=True)
    for class_value, count in zip(unique_classes, counts):
        if class_value in label_dict.values():
            class_name = [key for key, value in label_dict.items() if value == class_value][0]
            class_counts[class_name].append(filename)  # Append filename to the list
    return class_counts


In [ ]:
# train_classes_per_filenames = filename_classes( data_dir_train_mask+ 'masks/', LABEL_dict)

In [ ]:
# len(train_classes_per_filenames['background'])

In [ ]:
# def balance_classes (data_path,  LABEL_dict )
#     classes_filenames = filename_classes( data_path, LABEL_dict)
#     #step 1 : remove background from classes_filenames
#     #step 2 : detect low persentage classes (imbalanced classed)
#     #perform class balencing | make classes balanced
#     #step 3 for each file in the imbalanced class if the mask contains only 2 masks (including background) then augment that mask else if it contain other classes (other than the imbalanced class and the background) then check if the other classes present in the mask are 
# 
# 
#     # if the mask 

In [ ]:
# train_classes_per_filenames

In [ ]:

def balance_classes(data_path, LABEL_dict):
    classes_filenames = filename_classes(data_path, LABEL_dict)

    # Step 1: Remove background from classes_filenames
    #classes_filenames.pop('background')  # Remove background class

    # Step 2: Detect low percentage classes (imbalanced classes)
    class_counts = np.array([len(filenames) for filenames in classes_filenames.values()])
    majority_count = np.max(class_counts)*0.2
    majority_count_list = majority_count.tolist()  # Convert to list
    print(majority_count_list)
    imbalanced_classes = [
        class_name
        for class_name, filenames in classes_filenames.items()
            
            if int(len(filenames)) < int(majority_count_list)  # Access the first element (count is a list with a single element)
    ]
    print(f"Minority classes: {imbalanced_classes}")

    # Perform class balancing algorithm
    balanced_filenames = {}
    for class_name, filenames in classes_filenames.items():
        if class_name in imbalanced_classes:  # Apply oversampling or undersampling
            sampler = RandomOverSampler(sampling_strategy='not majority')  # Oversample minority classes
            # Or use undersampling:
            # sampler = RandomUnderSampler(sampling_strategy='majority')
            filenames_array = np.array(filenames).reshape(-1, 1)  # Reshape for sampling
            filenames_resampled, _ = sampler.fit_resample(filenames_array, filenames_array)
            balanced_filenames[class_name] = filenames_resampled.flatten().tolist()  # Convert back to list
        else:  # Keep majority classes as-is
            balanced_filenames[class_name] = filenames

    return balanced_filenames


In [ ]:
# balanced_filenames = balance_classes(data_dir_train_mask+ 'masks/', LABEL_dict)

In [ ]:
# for class_name, filenames in train_classes_per_filenames.items():
#     print(f"Class: {class_name} - Count: {len(filenames)}")


In [ ]:
 # for class_name, filenames in balanced_filenames.items():
 #    print(f"Class: {class_name} - Count: {len(filenames)}")

DATA GENERATORS

In [ ]:
# from collections import Counter
# 
# class_weights = {}
# masks = os.listdir(data_dir_train_mask + 'masks/')
# # Assuming data_dir_train_mask is a list containing paths to your training masks
# total_count = len(masks)  # Total number of masks
# 
# for mask_path in masks:
#     # Load the mask from the path (replace 'load_mask' with your loading function)
#     mask = cv2.imread(data_dir_train_mask + 'masks/'+mask_path, cv2.IMREAD_GRAYSCALE)
#     # Count the number of pixels for each class in the mask
#     class_counts = Counter(mask.flatten())
# 
#     # Calculate inverse frequency for each class
#     for class_label, count in class_counts.items():
#         class_weights[class_label] = class_weights.get(class_label, 0) + count
# 
# # Normalize class weights (optional but recommended)
# total_sum = sum(class_weights.values())
# for class_label, weight in class_weights.items():
#     class_weights[class_label] = weight / total_sum
# 
# inverted_weights = {class_label: 1.0 / weight for class_label, weight in class_weights.items()}
# 
# total_sum = sum(inverted_weights.values())
# for class_label, weight in inverted_weights.items():
#     inverted_weights[class_label] = weight / total_sum

In [ ]:
# inverted_weights

In [ ]:
def add_sample_weights(image, mask, class_weights):


  # Assuming class_weights is a dictionary with class labels as keys and weights as values
  #class_weights = {0: 2.0, 1: 2.0, 2: 1.0}  # Replace with your calculated weights

  # Normalize class weights (optional but recommended)
  # total_sum = sum(class_weights.values())
  # for class_label, weight in class_weights.items():
  #   class_weights[class_label] = weight / total_sum

  # One-hot encode the mask (optional, depending on your loss function)
  mask_one_hot = tf.one_hot(mask, depth=len(class_weights))

  # Create the sample weight image by multiplying each class channel with its weight
  sample_weights = mask_one_hot * tf.constant([class_weights[i] for i in range(len(class_weights))])

  # Reduce the sample weight image to a single channel (if needed)
  # sample_weights = tf.reduce_sum(sample_weights, axis=-1)  # Uncomment if needed

  return image, mask, sample_weights


In [ ]:
# image, mask= read_image_mask(data_dir_train_image + 'images/case_9_CT-slice074_x.png', data_dir_train_mask + 'masks/case_9_CT-slice074_x.png')

In [ ]:
# image, mask, sample_weights = add_sample_weights(image, mask, iverted_class_weights)

In [ ]:
# train_generator = create_segmentation_generator_train(data_dir_train_image, data_dir_train_mask, BATCH_SIZE_TRAIN)
# test_generator = create_segmentation_generator_test(data_dir_test_image, data_dir_test_mask, BATCH_SIZE_TEST)

In [ ]:
#show_dataset(train_generator, 3)

# Loading the dataset

In [126]:
from collections import Counter

class_weights = {}
masks = os.listdir(data_dir_train_mask + 'masks/')
# Assuming data_dir_train_mask is a list containing paths to your training masks
total_count = len(masks)  # Total number of masks

for mask_path in masks:
    # Load the mask from the path (replace 'load_mask' with your loading function)
    mask = cv2.imread(data_dir_train_mask + 'masks/'+mask_path, cv2.IMREAD_GRAYSCALE)
    # Count the number of pixels for each class in the mask
    class_counts = Counter(mask.flatten())

    # Calculate inverse frequency for each class
    for class_label, count in class_counts.items():
        class_weights[class_label] = class_weights.get(class_label, 0) + count

# Normalize class weights (optional but recommended)
total_sum = sum(class_weights.values())
for class_label, weight in class_weights.items():
    class_weights[class_label] = weight / total_sum

inverted_weights = {class_label: 1.0 / weight for class_label, weight in class_weights.items()}
# 
# total_sum = sum(inverted_weights.values())
# for class_label, weight in inverted_weights.items():
#     inverted_weights[class_label] = weight / total_sum

Exception ignored in: <function AtomicFunction.__del__ at 0x7fe17a2048b0>
Traceback (most recent call last):
  File "/home/aziz0220/anaconda3/envs/py310/lib/python3.10/site-packages/tensorflow/python/eager/polymorphic_function/atomic_function.py", line 226, in __del__
    self._bound_context.remove_function(self.name)
  File "/home/aziz0220/anaconda3/envs/py310/lib/python3.10/site-packages/tensorflow/python/eager/context.py", line 1435, in remove_function
    pywrap_tfe.TFE_ContextRemoveFunction(self._handle, name)
KeyboardInterrupt: 
Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7fe1b1d92080>>
Traceback (most recent call last):
  File "/home/aziz0220/anaconda3/envs/py310/lib/python3.10/site-packages/ipykernel/ipkernel.py", line 770, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
# inverted_weights

In [127]:
batch_size = 4
np.random.seed(42)
tf.random.set_seed(42)
create_dir("files")
(train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_x)}")
print("")

train_ds = tf_dataset_train(train_x, train_y, batch=batch_size)
valid_ds = tf_dataset_test(test_x, test_y, batch=batch_size)

Train: 3706/3706 - Valid: 0/0 - Test: 927/927



In [ ]:
def display(display_list):
  plt.figure(figsize=(15, 15))

  title = ['Input Image', 'True Mask', 'Predicted Mask']

  for i in range(len(display_list)):
    plt.subplot(1, len(display_list), i+1)
    plt.title(title[i])
    plt.imshow(tf.keras.utils.array_to_img(display_list[i]))
    plt.axis('off')
  plt.show()

In [ ]:
# for images, masks in train_ds.take(2):
#   sample_image, sample_mask = images[0], masks[0,:,:,:3]
#   display([sample_image, sample_mask])

#STEP 3 : BUILDING THE MODEL

In [73]:
# model = load_or_build_unet(os.path.join(dataset_path + "/filesfsdb", 'set1.h5'), input_shape, num_classes)

Model not found at /mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4/filesfsdb/set1.h5. Building a new model...
Input shape: (512, 512, 3), Number of classes: 31


# COMPILING THE MODEL

In [ ]:
# METRICS = [
#     keras.losses.CategoricalCrossentropy(name='crossentropy'),
#     keras.metrics.CategoricalAccuracy(name='accuracy'),
# ]
# model.compile(
#     loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#     optimizer=tf.keras.optimizers.Adam(lr),
#     metrics=METRICS  # Pass the list of metrics here
# )
# model.summary()

TRAINING THE MODEL

In [ ]:
# inverted_weights

In [ ]:
last_case_number = 0
dataset_path = '/mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4'
model = load_or_build_unet(os.path.join(dataset_path+f"/files_{last_case_number}", f"model_{last_case_number}_epoch.h5"), input_shape, num_classes)
last_case_number += 1
# METRICS = [
#     #keras.losses.CategoricalCrossentropy(name='crossentropy'),
#     keras.metrics.CategoricalAccuracy(name='accuracy'),
# ]
model.compile(
    # loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
    #loss="categorical_crossentropy",
    loss=tf.keras.losses.CategoricalFocalCrossentropy(),
    optimizer=tf.keras.optimizers.Adam(lr),
    # metrics=METRICS  # Pass the list of metrics here
)

In [ ]:
model.summary()

In [ ]:
!pip install pydot graphviz


In [ ]:
plot_model(model, to_file='model_plot.png', show_shapes=True, show_layer_names=True)

In [135]:
last_case_number = 3
while True:
    dataset_path = '/mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4'
    model = load_or_build_unet(os.path.join(dataset_path+f"/files_{last_case_number}", f"model_{last_case_number}_epoch.h5"), input_shape, num_classes)
    last_case_number += 1
    # METRICS = [
    #     #keras.losses.CategoricalCrossentropy(name='crossentropy'),
    #     keras.metrics.CategoricalAccuracy(name='accuracy'),
    # ]
    model.compile(
        # loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
        #loss="categorical_crossentropy",
        loss=tf.keras.losses.CategoricalFocalCrossentropy(),
        optimizer=tf.keras.optimizers.Adam(lr),
        # metrics=METRICS  # Pass the list of metrics here
    )
    model_path = os.path.join(dataset_path ,f"files_{last_case_number}")
    model_xx = os.path.join(model_path, f"model_{last_case_number}_epoch.h5")
    csv_path = os.path.join(dataset_path+f"/files_{last_case_number}", "data.csv")
    callbacks = [
    ModelCheckpoint(model_xx, verbose=1, save_best_only=True, monitor='val_loss'),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=5, min_lr=1e-7, verbose=1),
    CSVLogger(csv_path, append=True),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
    ]
    # model.fit(train_ds,
    #     validation_data=valid_ds,
    #     epochs=20,
    #     callbacks=callbacks
    #     # class_weight = inverted_weights
    # )
    # model.save(model_xx)
    SCORE = []
    hd_list = []
    for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
        """ Extract the name """
        name = x.split("/")[-1].split(".")[0]
        """ Reading the image """
        image = cv2.imread(x, cv2.IMREAD_COLOR)
        #image = cv2.resize(image, (image_w, image_h))
        #image = 1-image
        image_x = image
        #image = image.astype(np.uint8)
        image = image/255.0 ## (H, W, 3)
        #image = np.stack((image,)*3, axis=-1)
        image = np.expand_dims(image, axis=0) ## [1, H, W, 3]
        image = image.astype(np.float32)
        """ Reading the mask """
        mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
        #mask = cv2.resize(mask, (image_w, image_h))
        #mask = mask.astype(np.uint8)
        mask = mask.astype(np.int32)
        """ Prediction """
        pred = model.predict(image, verbose=0)[0]
        pred = np.argmax(pred, axis=-1) ## [0.1, 0.2, 0.1, 0.6] -> 3
        pred = pred.astype(np.int32)
         #cv2.imwrite("pred.png", pred * (255/31))
        # rgb_mask = grayscale_to_rgb(pred, rgb_codes)
        # cv2.imwrite("pred.png", rgb_mask)
        """ Save the results """
        save_image_path = dataset_path+ f"/results_{last_case_number}/{name}.png"
        save_results(image_x, mask, pred, save_image_path)
        """ Flatten the array """
        mask = mask.flatten()
        pred = pred.flatten()
        labels = [i for i in range(num_classes)]
        class_masks = []
        class_preds = []
        hd_list = np.empty(num_classes) 
        for c in range(num_classes):
            class_mask = (mask == c).astype(np.uint8)
            class_pred = (pred == c).astype(np.uint8)
            # Calculate Hausdorff Distance for each class
            hd = hausdorff_distance(class_pred, class_mask)
            hd_list[c] = hd  # Assign hd to the corresponding index in hd_list
        """ Calculating the metrics values """
        print(pred.shape)
        metrics = sg.write_metrics(labels=labels,  # exclude background if needed
                  gdth_img=mask,
                  pred_img=pred,
                  csv_file=dataset_path + f"/files_{last_case_number}/hd95.csv",  # save results to the csv_file 
                
                  metrics=['hd', 'hd95', 'msd'])
        f1_value = f1_score(mask, pred, labels=labels, average=None, zero_division=0)
        jac_value = jaccard_score(mask, pred, labels=labels, average=None, zero_division=0)
        SCORE.append([f1_value, jac_value, hd_list])
    score = np.array(SCORE)
    score = np.mean(score, axis=0)
    f = open(dataset_path + f"/files_{last_case_number}/score.csv", "w")
    f.write("Class,F1,Jaccard,HD95\n")
    l = ["Class", "F1", "Jaccard", "HD95"]
    print(f"{l[0]:15s} {l[1]:10s} {l[2]:10s} {l[3]:10s}")
    print("-"*35)
    for i in range(num_classes):
        class_name = classes[i]
        f1 = score[0, i]
        jac = score[1, i]
        hd95 = score[2, i]
        dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f} {hd95:1.5f}"
        print(dstr)
        f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f},{hd95:1.5f}\n")
    print("-"*35)
    class_mean = np.mean(score, axis=-1)
    class_name = "Mean"
    f1 = class_mean[0]
    jac = class_mean[1]
    hd95 = class_mean[2]
    dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f} - {hd95:1.5f}"
    print(dstr)
    f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f},{hd95:1.5f}\n")
    f.close()

Loaded existing model from /mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4/files_3/model_3_epoch.h5


  0%|          | 0/927 [00:00<?, ?it/s]

(262144,)


Exception: The dimension of gdth should be 2 or 3, but it is 1

In [ ]:
# test_classes = count_masks_per_class('/mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4/data_case1_10epochs_training_imbalenced_classes/test/masks/masks', LABEL_dict)

In [ ]:
# class_counts = {name: 0 for name in LABEL_dict.keys()}
# model_path = '/mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4/data_case1_10epochs_training_imbalenced_classes/files_4/model_4_epoch.h5'
# model = tf.keras.models.load_model(model_path)
# path = '/mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4/data_case1_10epochs_training_imbalenced_classes/test/images/images/'
# for x in os.listdir(path):    
#     """ Reading the image """
#     image = cv2.imread(path+x, cv2.IMREAD_COLOR)
#     image = image/255.0 
#     image = np.expand_dims(image, axis=0) ## [1, H, W, 3]
#     image = image.astype(np.float32)
#     """ Prediction """
#     pred = model.predict(image, verbose=0)[0]
#     pred = np.argmax(pred, axis=-1) ## [0.1, 0.2, 0.1, 0.6] -> 3
#     pred = pred.astype(np.int32)
#     class_counts = class_to_dict(pred, class_counts, LABEL_dict)   
# print("\nclasses per original masks : ",test_classes)
# print("\nclasses per predicted masks : ",class_counts)

# STEP 4 : PREDICTION & EVALUATION

In [ ]:
# np.random.seed(42)
# tf.random.set_seed(42)
# 
# """ Directory for storing files """
# create_dir("results")
# 
# """ Paths """
# dataset_path = '/mnt/c/Users/benam/Downloads/HaN-Seg/HaN-Seg/set_4'
# model_path = os.path.join(dataset_path + "/files_1", 'model_1_epoch.h5')
# 
# """ Loading the dataset """
# (train_x, train_y), (valid_x, valid_y), (test_x, test_y) = load_dataset(dataset_path)
# print(f"Train: {len(train_x)}/{len(train_y)} - Valid: {len(valid_x)}/{len(valid_y)} - Test: {len(test_x)}/{len(test_x)}")
# print("")
# 
# """ Load the model """
# model = tf.keras.models.load_model(model_path)
# 
# """ Prediction & Evaluation """
# SCORE = []
# for x, y in tqdm(zip(test_x, test_y), total=len(test_x)):
#     """ Extract the name """
#     name = x.split("/")[-1].split(".")[0]
# 
#     """ Reading the image """
#     image = cv2.imread(x, cv2.IMREAD_COLOR)
#     #image = cv2.resize(image, (image_w, image_h))
#     #image = 1-image
#     image_x = image
#     image = np.expand_dims(image, axis=0) ## [1, H, W, 3]
#     image = image.astype(np.float32)
#     
#     #image = np.stack((image,)*3, axis=-1)
#     #plt.imsave(dataset_path+f"/resultimg04/{name}.png", image, cmap='gray')
#     
#     image = np.expand_dims(image, axis=0) ## [1, H, W, 3]
#     
# 
#     """ Reading the mask """
#     mask = cv2.imread(y, cv2.IMREAD_GRAYSCALE)
#     #mask = cv2.resize(mask, (image_w, image_h))
#     #mask = np.stack((mask,)*3, axis=-1)
#     mask = mask.astype(np.int32)
# 
#     """ Prediction """
#     pred = model.predict(image, verbose=0)[0]
#     pred = np.argmax(pred, axis=-1) ## [0.1, 0.2, 0.1, 0.6] -> 3
#     pred = pred.astype(np.int32)
# 
#     #cv2.imwrite(dataset_path+f"/hhhhhhhhhhh/{name}.png", image_x )
#  
#     # rgb_mask = grayscale_to_rgb(pred, rgb_codes)
#     # cv2.imwrite("pred.png", rgb_mask)
# 
#     """ Save the results """
#     save_image_path = dataset_path+f"/results_1/{name}.png"
#     save_results(image_x, mask, pred, save_image_path)
# 
#     """ Flatten the array """
#     mask = mask.flatten()
#     pred = pred.flatten()
# 
#     labels = [i for i in range(num_classes)]
# 
#     """ Calculating the metrics values """
#     f1_value = f1_score(mask, pred, labels=labels, average=None, zero_division=0)
#     jac_value = jaccard_score(mask, pred, labels=labels, average=None, zero_division=0)
#     SCORE.append([f1_value, jac_value])
# 
# score = np.array(SCORE)
# score = np.mean(score, axis=0)
# 
# f = open(dataset_path+"/files_1/score.csv", "w")
# f.write("Class,F1,Jaccard\n")
# 
# l = ["Class", "F1", "Jaccard"]
# print(f"{l[0]:15s} {l[1]:10s} {l[2]:10s}")
# print("-"*35)
# 
# for i in range(num_classes):
#     class_name = classes[i]
#     f1 = score[0, i]
#     jac = score[1, i]
#     dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
#     print(dstr)
#     f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f}\n")
# 
# print("-"*35)
# class_mean = np.mean(score, axis=-1)
# class_name = "Mean"
# 
# f1 = class_mean[0]
# jac = class_mean[1]
# 
# dstr = f"{class_name:15s}: {f1:1.5f} - {jac:1.5f}"
# print(dstr)
# f.write(f"{class_name:15s},{f1:1.5f},{jac:1.5f}\n")
# 
# f.close()